# Hillslope Mesh Generation

This notebook illustrates generating a hillslope "open book" mesh using TINerator.

This process is done in two main parts:

1. Define the 2D planar quad mesh
2. Extrude to 3D into a hex mesh
3. Define face sets and write to disk

In [2]:
# Import TINerator and other required packages

import tinerator as tin
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
%matplotlib widget

ImportError: libpoppler.so.91: cannot open shared object file: No such file or directory

## 1. Define the 2D planar quad mesh

TINerator builds hillslope meshes in a similar fashion to NumPy's `meshgrid` function. That is to say, three components are required:

- `Z`: an $NxM$ matrix, where each cell in the matrix stores an elevation data and will become a singular point in the quad mesh
- `y`: the Y-vector, equal in length to the rows of `Z`, that defines the y-coordinate for each row
- `x`: the X-vector, equal in length to the columns of `Z`, that defines the x-coordinate for each column

Below, we define a function for generating a V-shaped $NxM$ grid. Jupyter widget integration allows one to manipulate these values and view the results directly.

In [3]:
def f(x, y, xv, zv, v_slope, y0, stream_slope):
    z = zv + v_slope*np.abs(x-xv) + (y-y0)*stream_slope
    return z

def make_mesh(
    stream_slope = 0.05,
    nx=11,
    ny=21,
    x_min=0.0,
    x_max=10.0,
    y_min=0.0,
    y_max=20.0,
    z_min=0.0,
    z_v=7.0,
    z_max=10.0,
):
    x_v = (x_max - x_min) / 2.
    dx = (x_max - x_min) / (nx - 1)
    dy = (y_max - y_min) / (ny - 1)
    
    v_slope = (z_max - z_v) / x_v
    
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)
    
    Z = f(X, Y, x_v, z_v, v_slope, y_min, stream_slope)
    
    return x, y, Z


# The external 'widgets.py' file will
#   compute `make_mesh` with widget functionality
# The parameter `s.xyZ()` returns the x, y, Z data
%run -i widgets.py
mm_widget = MakeMeshWidget()
display(mm_widget)

ModuleNotFoundError: No module named 'ipywidgets'

NameError: name 'MakeMeshWidget' is not defined

In [ ]:
x, y, Z = mm_widget.xyZ()

# Alternately, you can use the function directly:
# =============================== #
# x, y, Z = make_mesh(
#     nx=12,
#     ny=21,
#     stream_slope=0.05,
#     x_min=0.0,
#     x_max=10.0,
#     y_min=0.0,
#     y_max=20.0,
#     z_min=0.0,
#     z_max=10.0,
#     z_v=7.0,
# )

quad_mesh = tin.meshing.create_hillslope_mesh(
    Z, x_coords=x, y_coords=y
)
quad_mesh.view()

## 2. Extrude the 2D mesh into a 3D hex mesh

We define the subsurface by extruding in the `-Z` direction. The `extrude_mesh` function takes in a list of one or more tuples defining the *layer schema*, in this case:

```python
("snapped", z_min, n_z, 1)
```

Where:

* `"snapped"` => flat-bottomed layer (not terrain following)
* `z_min` => the Z value to 'snap' all nodes at the bottom of the layer to
* `n_z` => the number of cells deep that compose the layer
* `1` => the "material ID" value of the layer

In [ ]:
z_min = 0.0
n_z = 11

layers = [("snapped", z_min, n_z, 1),]
hex_mesh = tin.meshing.extrude_mesh(quad_mesh, layers)
hex_mesh.view()

In [ ]:
x_min, x_max = np.min(x), np.max(x)
y_min, y_max = np.min(y), np.max(y)
z_min, z_max = 0, np.max(Z)

x_center = x_min + (x_max - x_min) / 2.

fractures = [
    [[x_min, y_min, z_min], [x_max, y_max, z_max]],
    [[x_center, y_min, 8.5], [x_center, y_max, 8.0]],
    [[x_min, y_min, z_max / 2], [x_max, y_max / 2, z_max / 2]],
]

for (i, fracture) in enumerate(fractures):
    cell_ids = hex_mesh.get_cells_along_line(fracture[0], fracture[1])
    hex_mesh.set_cell_materials(cell_ids, 5 + i)

hex_mesh.view(active_scalar="material_id")

### 3. Create face sets and write to ExodusII file

Face sets are defined as containing one or more faces from the *surface mesh*; or, the exterior of the extruded hex mesh.

Below, we are extracting faces from the top of the mesh, along with faces that can be grouped by the face normal vector.

In [ ]:
surface_mesh = hex_mesh.surface_mesh()

In [ ]:
sets = []

sets.extend([
    surface_mesh.top_faces,
    surface_mesh.bottom_faces,
    surface_mesh.side_faces
])

# We can also get north, south, west, east, up, down-facing cell faces:
# ====================================================================
#sets.extend(
#    s for s in surface_mesh.from_cell_normals()
#    if s.name in ["north", "west", "east", "south"]
#)

surface_mesh.validate_sets(sets)

hex_mesh.view(sets=sets)
hex_mesh.save("Open-Book-3D.exo", sets=sets)